In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

#DATA_DIR = "/kaggle/input/predict-energy-behavior-of-prosumers/"
DATA_DIR = "./"

df_data = pd.read_csv(DATA_DIR + "train.csv")
df_client = pd.read_csv(DATA_DIR + "client.csv")
df_historical_weather = pd.read_csv(DATA_DIR + "historical_weather.csv")
df_forecast_weather = pd.read_csv(DATA_DIR + "forecast_weather.csv")
df_electricity_prices = pd.read_csv(DATA_DIR + "electricity_prices.csv")
df_gas_prices = pd.read_csv(DATA_DIR + "gas_prices.csv")
df_weather_station_to_county_mapping = pd.read_csv(DATA_DIR + "weather_station_to_county_mapping.csv")

In [2]:
def upsample_daily_to_hourly(df: pd.DataFrame, date_col: str) -> pd.DataFrame:
    # unsample (process) client data. Client data is reported daily, but we need it hourly.
    # So we take ever row (1 day) and duplicate (explode) it into 24 rows
    df_hourly = df.copy(deep=True) # deep copy to not make changes to dataframe from parameter
    df_hourly[date_col] = pd.to_datetime(df_hourly[date_col])

    # create a column of lists, where each list contains [00:00, 01:00, ... 23:00] for that day
    df_hourly['datetime'] = df_hourly[date_col].apply(lambda x: [x + pd.Timedelta(hours=i) for i in range(24)])
    df_hourly = df_hourly.explode('datetime')

    # drop the original daily date column as it's no longer needed
    df_hourly = df_hourly.drop(columns=[date_col])
    return df_hourly


# nb! needs more testing as this method was generated by AI.
def process_forecast_weather(df_forecast: pd.DataFrame, location_map: dict) -> pd.DataFrame:
    """
    Cleans, aggregates, and pivots forecast weather data.

    Logic:
    1. Map Lat/Lon to County ID.
    2. Convert 'Origin Time' (when forecast was made) to 'Target Time' (when weather happens).
    3. Group forecasts into 'batches' (Day 1 forecast vs Day 2 forecast).
    4. Average the values per county/hour/batch.
    5. Pivot so batches become columns (e.g., temperature_1, temperature_2).
    """
    # Safety Copy
    df = df_forecast.copy(deep=True)

    # 1. Map Coordinates to County
    # We use the dictionary passed in from the main function
    df['county'] = [location_map.get((x, y), -1) for x, y in zip(df['latitude'], df['longitude'])]

    # Filter out valid locations only
    df = df[df['county'] != -1]

    # 2. Calculate Target Time
    # Standardize origin time to 02:00:00 (removes minute/second noise)
    df['origin_datetime'] = pd.to_datetime(df['origin_datetime'])
    df['origin_datetime'] = pd.to_datetime(df['origin_datetime'].dt.date.astype(str) + ' 02:00:00')

    # Target Time = Origin + Hours Ahead
    df['forecast_datetime'] = df['origin_datetime'] + pd.to_timedelta(df['hours_ahead'], unit='h')

    # We don't need origin time anymore
    df.drop(columns=['origin_datetime'], inplace=True)

    # 3. Create "Batches" (cumcount) todo:rename
    # A batch represents how far out the forecast is (Day 1 vs Day 2)
    # (hours_ahead - 1) // 24 + 1 results in: 1 for 0-24h, 2 for 25-48h
    df['cumcount'] = (df['hours_ahead'] - 1) // 24 + 1

    # 4. Aggregate (Mean) by County, Time, and Batch
    # Identify feature columns (exclude IDs and Time)
    exclude_cols = ['latitude', 'longitude', 'hours_ahead', 'forecast_datetime', 'cumcount', 'county', 'data_block_id']
    feature_cols = [col for col in df.columns if col not in exclude_cols]

    agg_dict = {col: 'mean' for col in feature_cols}
    # We must keep 'cumcount' in the groupby keys, so we don't aggregate it

    df_grouped = df.groupby(['county', 'forecast_datetime', 'cumcount']).agg(agg_dict)

    # 5. Pivot (Unstack)
    # Moves 'cumcount' from a row index to a column suffix
    df_pivoted = df_grouped.unstack(level=-1)

    # Flatten MultiIndex columns: ('temperature', 1) -> 'temperature_1'
    df_pivoted.columns = [f'{col[0]}_{col[1]}' for col in df_pivoted.columns]

    df_pivoted.reset_index(inplace=True)
    df_pivoted.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)

    # Handle missing values (if a forecast is missing, fill with 0 or strictly manage it)
    df_pivoted.fillna(0, inplace=True)

    return df_pivoted

def process_historical_weather(df_historical: pd.DataFrame, location_map: dict) -> pd.DataFrame:
    #compose historical weather data (averaging stations per county).
    df = df_historical.copy(deep=True)
    df['datetime'] = pd.to_datetime(df['datetime'])

    df['county'] = [location_map.get((x, y), -1) for x, y in zip(df['latitude'], df['longitude'])]
    df = df[df['county'] != -1]

    exclude_cols = ['latitude', 'longitude', 'datetime', 'county', 'data_block_id']
    agg_dict = {col: 'mean' for col in df.columns if col not in exclude_cols}
    df_grouped = df.groupby(['county', 'datetime']).agg(agg_dict)
    df_grouped.reset_index(inplace=True)

    return df_grouped

def generate_features(
    df_data: pd.DataFrame,
    df_client: pd.DataFrame,
    df_gas_prices: pd.DataFrame,
    df_electricity_prices: pd.DataFrame,
    df_historical_weather: pd.DataFrame,
    df_forecast_weather: pd.DataFrame,
    df_weather_station_to_county_mapping: pd.DataFrame,
    train_start = '2021-09-01 11:00:00'
):
  # The weather data uses Latitude/Longitude, but the energy data uses "Counties".
  # We need a dictionary to translate coordinates into county IDs so we can join them later.
  # dictionary: {(lat, lon) -> county_id}
  df_weather_station_to_county_mapping = df_weather_station_to_county_mapping[
      df_weather_station_to_county_mapping.notnull().all(axis=1)
    ].sort_values(by="county")
  result_dict = dict(zip(
      zip(
        round(df_weather_station_to_county_mapping['latitude'],1),
        round(df_weather_station_to_county_mapping['longitude'],1)),
      df_weather_station_to_county_mapping['county']))
  df_historical_weather = df_historical_weather[df_historical_weather['datetime'] >= train_start]

  # ---------------------- client data -----------------------
  df_client_hourly = upsample_daily_to_hourly(df_client, date_col='date')
  if 'data_block_id' in df_client_hourly.columns:
    df_client_hourly.drop(columns=['data_block_id'], inplace=True)

  # merge into our main dataframe
  df_data['datetime'] = pd.to_datetime(df_data['datetime'])
  df_data = df_data.merge(df_client_hourly, on=['county','product_type','is_business','datetime'], how='left')
  # Filter data to ensure we don't go past the available client data
  client_end_date = df_client_hourly['datetime'].max()
  df_data = df_data[df_data['datetime'] <= client_end_date]

  # --------------- gas prices -------------------
  df_gas_hourly = upsample_daily_to_hourly(df_gas_prices, date_col='forecast_date')
  cols_to_drop = ['origin_date', 'data_block_id']
  df_gas_hourly.drop(columns=[c for c in cols_to_drop if c in df_gas_hourly.columns], inplace=True)

  gas_end_date = df_gas_hourly['datetime'].max()
  df_data = df_data[df_data['datetime'] <= gas_end_date]

  df_data = df_data.merge(df_gas_hourly, on=['datetime'], how='left')

  #--------------- electricity (already hourly) ------------------------------
  df_electricity_prices_try = df_electricity_prices.copy(deep=True)
  if 'origin_date' in df_electricity_prices_try.columns:
    df_electricity_prices_try.drop(columns=['origin_date'], inplace=True)
  if 'data_block_id' in df_electricity_prices_try.columns:
    df_electricity_prices_try.drop(columns=['data_block_id'], inplace=True)
  df_electricity_prices_try['forecast_date'] = pd.to_datetime(df_electricity_prices_try['forecast_date'])
  df_electricity_prices_try.rename(columns={"forecast_date": "datetime"}, inplace=True)
  df_data = df_data.merge(df_electricity_prices_try, on=['datetime'], how='left')

  # --- forecast weather ---
  df_forecast_processed = process_forecast_weather(df_forecast_weather, result_dict)
  df_data = df_data.merge(df_forecast_processed, on=['county', 'datetime'], how='left')


  # --- process historical weather ---
  df_weather_processed = process_historical_weather(df_historical_weather, result_dict)
  df_data = df_data.merge(df_weather_processed, on=['county', 'datetime'], how='left')

  return df_data

In [3]:
# this is to get rid of NaN values
min_end_date = min([
    pd.to_datetime(df_client['date']).max(),
    pd.to_datetime(df_gas_prices['forecast_date']).max(),
    pd.to_datetime(df_electricity_prices['forecast_date']).max(),
    pd.to_datetime(df_historical_weather['datetime']).max(),
    pd.to_datetime(df_forecast_weather['origin_datetime']).max()
])

print(f"Filtering all data to common end date: {min_end_date}")

max_datetime = min_end_date + pd.Timedelta(hours=23)
df_data = df_data[pd.to_datetime(df_data['datetime']) <= max_datetime].copy()

print(f"df_data: {df_data['datetime'].min()} → {df_data['datetime'].max()}")
print(f"Rows: {len(df_data):,}")

Filtering all data to common end date: 2023-05-29 00:00:00
df_data: 2021-09-01 00:00:00 → 2023-05-29 23:00:00
Rows: 2,012,112


In [4]:
combined_df = generate_features(
    df_data,
    df_client,
    df_gas_prices,
    df_electricity_prices,
    df_historical_weather,
    df_forecast_weather,
    df_weather_station_to_county_mapping)

In [ ]:
combined_df.head()

In [ ]:
combined_df.columns

In [6]:
# Analyze NaN values
nan_summary = combined_df.isnull().sum()
nan_columns = nan_summary[nan_summary > 0].sort_values(ascending=False)

print("="*70)
print("NaN ANALYSIS")
print("="*70)

if len(nan_columns) == 0:
    print("NO NaN VALUES FOUND!")
else:
    print(f"Found NaN values in {len(nan_columns)} columns:\n")
    for col, count in nan_columns.items():
        pct = (count / len(combined_df)) * 100
        print(f"   {col}: {count:,} ({pct:.2f}%)")
    
    print(f"\nTotal NaN: {combined_df.isnull().sum().sum():,}")
    print(f"Total cells: {combined_df.size:,}")
    print(f"NaN percentage: {(combined_df.isnull().sum().sum() / combined_df.size) * 100:.2f}%")


print(f"Dataset shape: {combined_df.shape}")
print(f"Date range: {combined_df['datetime'].min()} → {combined_df['datetime'].max()}")


NaN ANALYSIS
Found NaN values in 40 columns:

   winddirection_10m: 31,848 (1.58%)
   shortwave_radiation: 31,848 (1.58%)
   cloudcover_high: 31,848 (1.58%)
   windspeed_10m: 31,848 (1.58%)
   direct_solar_radiation: 31,848 (1.58%)
   diffuse_radiation: 31,848 (1.58%)
   cloudcover_mid: 31,848 (1.58%)
   cloudcover_low: 31,848 (1.58%)
   rain: 31,848 (1.58%)
   snowfall: 31,848 (1.58%)
   cloudcover_total: 31,848 (1.58%)
   surface_pressure: 31,848 (1.58%)
   temperature: 31,848 (1.58%)
   dewpoint: 31,848 (1.58%)
   temperature_1: 30,888 (1.54%)
   temperature_2: 30,888 (1.54%)
   snowfall_2: 30,888 (1.54%)
   snowfall_1: 30,888 (1.54%)
   cloudcover_high_2: 30,888 (1.54%)
   cloudcover_high_1: 30,888 (1.54%)
   dewpoint_1: 30,888 (1.54%)
   dewpoint_2: 30,888 (1.54%)
   cloudcover_low_1: 30,888 (1.54%)
   cloudcover_low_2: 30,888 (1.54%)
   10_metre_u_wind_component_2: 30,888 (1.54%)
   10_metre_u_wind_component_1: 30,888 (1.54%)
   surface_solar_radiation_downwards_2: 30,888 (1.54%)

In [9]:
# Defining column groups
weather_cols = [
    'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 
    'cloudcover_mid', 'cloudcover_total', 'windspeed_10m', 
    'winddirection_10m', 'rain', 'snowfall', 'surface_pressure',
    'shortwave_radiation', 'direct_solar_radiation', 'diffuse_radiation'
]

forecast_cols = [col for col in combined_df.columns if col.endswith('_1') or col.endswith('_2')]

print("Handling NaN values...")
print(f"Before: {combined_df.isnull().sum().sum():,} NaN values")

# Sorting by county and time
combined_df = combined_df.sort_values(['county', 'datetime']).reset_index(drop=True)

# 1. Interpolate weather data per county
for col in weather_cols + forecast_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df.groupby('county')[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )

# 2. Fill remaining NaN with forward/backward fill
combined_df[weather_cols] = combined_df.groupby('county')[weather_cols].ffill().bfill()
combined_df[forecast_cols] = combined_df.groupby('county')[forecast_cols].ffill().bfill()

# 3. Handle electricity prices
combined_df['euros_per_mwh'] = combined_df['euros_per_mwh'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')

# 4. Final safety: fill any remaining with 0 (very rare)
weather_and_forecast = [col for col in weather_cols + forecast_cols if col in combined_df.columns]
combined_df[weather_and_forecast] = combined_df[weather_and_forecast].fillna(0)
combined_df['euros_per_mwh'] = combined_df['euros_per_mwh'].fillna(combined_df['euros_per_mwh'].mean())

print(f"After: {combined_df.isnull().sum().sum():,} NaN values")
print(f"Remaining NaN in 'target': {combined_df['target'].isnull().sum()} (expected for test data)")

Handling NaN values...
Before: 794 NaN values


C:\Users\merlink0\AppData\Local\Temp\ipykernel_24552\2776406172.py:29: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df['euros_per_mwh'] = combined_df['euros_per_mwh'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')


After: 528 NaN values
Remaining NaN in 'target': 528 (expected for test data)


In [12]:
!pip install xgboost scikit-learn

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   - -------------------------------------- 2.1/72.0 MB 11.7 MB/s eta 0:00:06
   -- ------------------------------------- 4.7/72.0 MB 11.6 MB/s eta 0:00:06
   --- ------------------------------------ 7.1/72.0 MB 11.6 MB/s eta 0:00:06
   ----- ---------------------------------- 9.2/72.0 MB 11.0 MB/s eta 0:00:06
   ------ --------------------------------- 11.5/72.0 MB 11.1 MB/s eta 0:00:06
   ------- -------------------------------- 14.2/72.0 MB 11.1 MB/s eta 0:00:06
   --------- ------------------------------ 16.5/72.0 MB 11.1 MB/s eta 0:00:05
   ---------- ----------------------------- 18.9/72.0 MB 11.2 MB/s eta 0:00:05
   ----------- ---------------------------- 21.5/72.0 MB 11.2 MB/s eta 0:00:05
   ------------- -------------------------- 23.9/72.0 MB 11.3 MB/s eta 0:00:05
   -------------- ------------------------- 26.2/72.0 MB 11.3 MB/s eta 0:00:05
   --------------- ------------------------ 28.6/72.0 MB 11.3 MB/

In [14]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# ============================================================================
# 1. DATA PREPARATION - MINIMAL FEATURES
# ============================================================================

# Define minimal feature set (only most important)
minimal_features = [
    'county', 'is_business', 'product_type', 'is_consumption',
    'eic_count', 'installed_capacity',
    'euros_per_mwh',  # Electricity price
    'lowest_price_per_mwh', 'highest_price_per_mwh',  # Gas prices
    'temperature', 'cloudcover_total', 'rain',  # Basic weather
    'temperature_1', 'temperature_2',  # Forecast temps
]

# Split train/test
train_df = combined_df[combined_df['target'].notna()].copy()
test_df = combined_df[combined_df['target'].isna()].copy()

print(f"Train rows: {len(train_df):,}")
print(f"Test rows: {len(test_df):,}")

# Sample train data to save memory (use 30% of data)
train_sample = train_df.sample(frac=0.3, random_state=42)
print(f"Sampled train rows: {len(train_sample):,} (30% of full data)")

# Prepare features
X = train_sample[minimal_features].copy()
y = train_sample['target'].copy()

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nX_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

# ============================================================================
# 2. XGBOOST MODEL - MEMORY EFFICIENT
# ============================================================================

# Minimal parameters for low memory usage
params = {
    'objective': 'reg:squarederror',
    'max_depth': 5,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'n_jobs': -1,
    'tree_method': 'hist',
    'early_stopping_rounds': 10,  # ← Siia sisse!
}

print("\nTraining XGBoost...")
model = xgb.XGBRegressor(**params)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=10
)


# ============================================================================
# 3. EVALUATION
# ============================================================================

# Predictions
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

# Metrics
train_mae = mean_absolute_error(y_train, y_pred_train)
val_mae = mean_absolute_error(y_val, y_pred_val)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))

print("\n" + "="*70)
print("MODEL PERFORMANCE")
print("="*70)
print(f"Train MAE:  {train_mae:.4f}")
print(f"Val MAE:    {val_mae:.4f}")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Val RMSE:   {val_rmse:.4f}")
print("="*70)

# Feature importance
import pandas as pd
feature_importance = pd.DataFrame({
    'feature': minimal_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

# ============================================================================
# 4. PREDICT ON TEST (if needed)
# ============================================================================

# Uncomment if you want test predictions
# X_test = test_df[minimal_features].copy()
# test_predictions = model.predict(X_test)
# test_df['predicted_target'] = test_predictions

Train rows: 2,011,584
Test rows: 528
Sampled train rows: 603,475 (30% of full data)

X_train shape: (482780, 14)
X_val shape: (120695, 14)

Training XGBoost...
[0]	validation_0-rmse:820.59801
[10]	validation_0-rmse:446.21080
[20]	validation_0-rmse:344.18446
[30]	validation_0-rmse:317.51043
[40]	validation_0-rmse:302.94427
[50]	validation_0-rmse:296.57800
[60]	validation_0-rmse:292.05050
[70]	validation_0-rmse:287.56444
[80]	validation_0-rmse:285.27242
[90]	validation_0-rmse:283.33125
[99]	validation_0-rmse:281.38850

MODEL PERFORMANCE
Train MAE:  108.7295
Val MAE:    110.8259
Train RMSE: 270.5967
Val RMSE:   281.3885

Top 10 Most Important Features:
               feature  importance
1          is_business    0.360651
3       is_consumption    0.216104
5   installed_capacity    0.168930
4            eic_count    0.102530
0               county    0.034795
9          temperature    0.028011
12       temperature_1    0.019687
6        euros_per_mwh    0.019338
10    cloudcover_total    0

In [17]:
# ============================================================================
# IMPROVED FEATURE ENGINEERING
# ============================================================================

# Time features
combined_df['hour'] = pd.to_datetime(combined_df['datetime']).dt.hour
combined_df['day_of_week'] = pd.to_datetime(combined_df['datetime']).dt.dayofweek
combined_df['month'] = pd.to_datetime(combined_df['datetime']).dt.month
combined_df['is_weekend'] = combined_df['day_of_week'].isin([5, 6]).astype(int)

# Lag features (historical consumption)
combined_df = combined_df.sort_values(['county', 'product_type', 'is_business', 'datetime'])
combined_df['target_lag_24h'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].shift(24)
combined_df['target_lag_168h'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].shift(168)

# Improved feature set
improved_features = [
    'county', 'is_business', 'product_type', 'is_consumption',
    'eic_count', 'installed_capacity',
    'euros_per_mwh', 'lowest_price_per_mwh', 'highest_price_per_mwh',
    'temperature', 'cloudcover_total', 'rain',
    'temperature_1', 'temperature_2',
    'hour', 'day_of_week', 'month', 'is_weekend',
    'target_lag_24h', 'target_lag_168h',
]

# Split
train_df = combined_df[combined_df['target'].notna()].copy()
test_df = combined_df[combined_df['target'].isna()].copy()

# Remove rows with NaN in lag features (first 168 hours)
train_df = train_df.dropna(subset=['target_lag_24h', 'target_lag_168h'])

# Sample 50% now
train_sample = train_df.sample(frac=0.5, random_state=42)
print(f"Sampled train rows: {len(train_sample):,}")

X = train_sample[improved_features].copy()
y = train_sample['target'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ============================================================================
# IMPROVED MODEL
# ============================================================================

params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'gamma': 0.1,
    'random_state': 42,
    'n_jobs': -1,
    'tree_method': 'hist',
    'early_stopping_rounds': 20,
}

print("\nTraining Improved XGBoost...")
model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=20)

y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

# Metrics
train_mae = mean_absolute_error(y_train, y_pred_train)
val_mae = mean_absolute_error(y_val, y_pred_val)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))

print("\n" + "="*70)
print("MODEL PERFORMANCE")
print("="*70)
print(f"Train MAE:  {train_mae:.4f}")
print(f"Val MAE:    {val_mae:.4f}")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Val RMSE:   {val_rmse:.4f}")
print("="*70)

# Feature importance
import pandas as pd
feature_importance = pd.DataFrame({
    'feature': improved_features,  # ← MUUTUS: minimal_features → improved_features
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

# ============================================================================
# 4. PREDICTIONS ON TEST DATA
# ============================================================================

print("\n" + "="*70)
print("MAKING PREDICTIONS ON TEST DATA")
print("="*70)

# Prepare test features
X_test = test_df[improved_features].copy()

# Check for NaN in test features
test_nan_count = X_test.isnull().sum().sum()
if test_nan_count > 0:
    print(f"⚠️ Warning: {test_nan_count} NaN values in test features")
    print("Filling NaN with 0...")
    X_test = X_test.fillna(0)

# Make predictions
test_predictions = model.predict(X_test)

# Add predictions to test dataframe
test_df['predicted_target'] = test_predictions

print(f"\nTest predictions made: {len(test_predictions)}")
print(f"Prediction range: {test_predictions.min():.2f} to {test_predictions.max():.2f}")
print(f"Mean prediction: {test_predictions.mean():.2f}")

# Show sample predictions
print("\nSample predictions:")
print(test_df[['datetime', 'county', 'product_type', 'is_business', 'predicted_target']].head(10))

# ============================================================================
# 5. SAVE PREDICTIONS
# ============================================================================

# Create submission file
submission = test_df[['row_id', 'predicted_target']].copy()
submission.columns = ['row_id', 'target']

print("\n" + "="*70)
print("SUBMISSION FILE")
print("="*70)
print(submission.head())
print(f"\nTotal predictions: {len(submission)}")


Sampled train rows: 999,468

Training Improved XGBoost...
[0]	validation_0-rmse:880.31271
[20]	validation_0-rmse:412.01673
[40]	validation_0-rmse:268.37377
[60]	validation_0-rmse:220.24941
[80]	validation_0-rmse:201.45002
[100]	validation_0-rmse:191.31822
[120]	validation_0-rmse:182.78789
[140]	validation_0-rmse:175.70998
[160]	validation_0-rmse:170.49251
[180]	validation_0-rmse:165.43575
[199]	validation_0-rmse:161.18926

MODEL PERFORMANCE
Train MAE:  67.1620
Val MAE:    68.1940
Train RMSE: 155.1686
Val RMSE:   161.1893

Top 10 Most Important Features:
               feature  importance
18      target_lag_24h    0.435558
19     target_lag_168h    0.127719
3       is_consumption    0.119138
1          is_business    0.046995
5   installed_capacity    0.041308
15         day_of_week    0.031271
12       temperature_1    0.027415
0               county    0.023491
10    cloudcover_total    0.020867
4            eic_count    0.020273

MAKING PREDICTIONS ON TEST DATA

Test predictions made

In [18]:
# ============================================================================
# ADDITIONAL IMPROVEMENTS
# ============================================================================

# 1. ADD MORE LAG FEATURES
combined_df = combined_df.sort_values(['county', 'product_type', 'is_business', 'datetime'])

# Rolling averages (sujuvamad trendid)
combined_df['target_rolling_24h_mean'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].transform(
    lambda x: x.shift(1).rolling(window=24, min_periods=1).mean()
)

combined_df['target_rolling_168h_mean'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].transform(
    lambda x: x.shift(1).rolling(window=168, min_periods=1).mean()
)

# Lag 48h ja 72h
combined_df['target_lag_48h'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].shift(48)
combined_df['target_lag_72h'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].shift(72)

# 2. INTERACTION FEATURES
combined_df['temp_x_hour'] = combined_df['temperature'] * combined_df['hour']
combined_df['price_x_hour'] = combined_df['euros_per_mwh'] * combined_df['hour']
combined_df['capacity_x_consumption'] = combined_df['installed_capacity'] * combined_df['is_consumption']

# 3. IMPROVED FEATURE SET
improved_features_v2 = [
    'county', 'is_business', 'product_type', 'is_consumption',
    'eic_count', 'installed_capacity',
    'euros_per_mwh', 'lowest_price_per_mwh', 'highest_price_per_mwh',
    'temperature', 'cloudcover_total', 'rain',
    'temperature_1', 'temperature_2',
    'hour', 'day_of_week', 'month', 'is_weekend',
    'target_lag_24h', 'target_lag_48h', 'target_lag_72h', 'target_lag_168h',
    'target_rolling_24h_mean', 'target_rolling_168h_mean',
    'temp_x_hour', 'price_x_hour', 'capacity_x_consumption',
]

# Split
train_df = combined_df[combined_df['target'].notna()].copy()
test_df = combined_df[combined_df['target'].isna()].copy()

# Remove NaN rows
train_df = train_df.dropna(subset=improved_features_v2)

# Use 70% data now (more is better with more features)
train_sample = train_df.sample(frac=0.7, random_state=42)
print(f"Sampled train rows: {len(train_sample):,}")

X = train_sample[improved_features_v2].copy()
y = train_sample['target'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ============================================================================
# IMPROVED MODEL V2
# ============================================================================

params = {
    'objective': 'reg:squarederror',
    'max_depth': 7,  # 6 → 7
    'learning_rate': 0.03,  # 0.05 → 0.03 (aeglasem)
    'n_estimators': 300,  # 200 → 300
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 5,  # 3 → 5
    'gamma': 0.2,  # 0.1 → 0.2
    'reg_alpha': 0.1,  # L1 regularization
    'reg_lambda': 1.0,  # L2 regularization
    'random_state': 42,
    'n_jobs': -1,
    'tree_method': 'hist',
    'early_stopping_rounds': 30,  # 20 → 30
}

print("\nTraining Improved XGBoost V2...")
model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=20)

# ============================================================================
# EVALUATION
# ============================================================================

y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

train_mae = mean_absolute_error(y_train, y_pred_train)
val_mae = mean_absolute_error(y_val, y_pred_val)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))

print("\n" + "="*70)
print("MODEL PERFORMANCE V2")
print("="*70)
print(f"Train MAE:  {train_mae:.4f}")
print(f"Val MAE:    {val_mae:.4f}")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Val RMSE:   {val_rmse:.4f}")
print("="*70)

# Feature importance
feature_importance = pd.DataFrame({
    'feature': improved_features_v2,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 15 Most Important Features:")
print(feature_importance.head(15))

# ============================================================================
# PREDICTIONS WITH NON-NEGATIVE CONSTRAINT
# ============================================================================

X_test = test_df[improved_features_v2].copy()
X_test = X_test.fillna(0)

test_predictions = model.predict(X_test)

# FIX NEGATIVE PREDICTIONS
test_predictions = np.maximum(test_predictions, 0)  # ← OLULINE!

test_df['predicted_target'] = test_predictions

print("\n" + "="*70)
print("TEST PREDICTIONS (NON-NEGATIVE)")
print("="*70)
print(f"Predictions made: {len(test_predictions)}")
print(f"Prediction range: {test_predictions.min():.2f} to {test_predictions.max():.2f}")
print(f"Mean prediction: {test_predictions.mean():.2f}")
print(f"Negative predictions fixed: {(model.predict(X_test) < 0).sum()}")

# Sample
print("\nSample predictions:")
print(test_df[['datetime', 'county', 'product_type', 'predicted_target']].head(10))

# ============================================================================
# SAVE SUBMISSION
# ============================================================================

submission = test_df[['row_id', 'predicted_target']].copy()
submission.columns = ['row_id', 'target']
submission.to_csv('submission_v2.csv', index=False)
print("\n✅ Saved to: submission_v2.csv")

Sampled train rows: 1,398,516

Training Improved XGBoost V2...
[0]	validation_0-rmse:902.91934
[20]	validation_0-rmse:524.58631
[40]	validation_0-rmse:326.07719
[60]	validation_0-rmse:230.20701
[80]	validation_0-rmse:186.34827
[100]	validation_0-rmse:165.98828
[120]	validation_0-rmse:155.88461
[140]	validation_0-rmse:149.58479
[160]	validation_0-rmse:145.30097
[180]	validation_0-rmse:141.79791
[200]	validation_0-rmse:139.60287
[220]	validation_0-rmse:137.65919
[240]	validation_0-rmse:135.98184
[260]	validation_0-rmse:134.36455
[280]	validation_0-rmse:132.67342
[299]	validation_0-rmse:131.23348

MODEL PERFORMANCE V2
Train MAE:  42.1592
Val MAE:    43.9662
Train RMSE: 118.0630
Val RMSE:   131.2335

Top 15 Most Important Features:
                     feature  importance
19            target_lag_48h    0.494996
18            target_lag_24h    0.207594
20            target_lag_72h    0.067318
26    capacity_x_consumption    0.056321
3             is_consumption    0.044529
22   target_roll